In [1]:
# Import Library
import pandas as pd
import os
import re
import pymongo
from pymongo import MongoClient
from dotenv import load_dotenv

from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA

In [2]:
# Load Environment Variables
load_dotenv(override=True)
MONGODB_URI=os.getenv("MONGODB_URI")
OPENAI_KEY=os.getenv("OPENAI_API_KEY")

In [3]:
# Initialize Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_KEY,
    dimensions=1536
)

In [4]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['finalproject_db']['faq']

In [5]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
    text_key="text"
)

In [6]:
# Optimized Professional Prompt Template
PROFESSIONAL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Resmi Allo Bank Indonesia.
    Tugas Anda adalah menjawab pertanyaan berdasarkan data FAQ resmi dari website Allo Bank.

    Tanggung jawab utama:
    Memberikan jawaban akurat dan informatif sesuai dengan kategori FAQ: umum, akun, produk, layanan, proses pengaduan nasabah, dan registrasi.
    Menyampaikan informasi dengan bahasa profesional, jelas, dan mudah dipahami.
    Memberikan panduan teknis (jika diperlukan) dalam format numerik atau langkah-langkah berurutan.
    Menanggapi keluhan atau kendala nasabah dengan solusi yang relevan dari FAQ.

    Aturan jawaban:
    1. Jawab hanya berdasarkan informasi yang tersedia dalam konteks FAQ resmi.
    2. Jika pertanyaan tidak relevan atau tidak tercakup dalam FAQ, jawab dengan sopan bahwa Anda tidak memiliki informasi tersebut.
    3. Jangan mengarang jawaban atau memberikan informasi di luar dokumen sumber.
    4. Jangan mencantumkan link eksternal, tautan promosi, atau informasi tambahan di luar konteks.
    5. Gunakan format poin atau numerik bila menjelaskan prosedur atau langkah-langkah.

    Konteks resmi:
    {context}

    Pertanyaan: {question}

    Jawaban profesional:
    """
)

In [7]:
# Model Configuration dengan GPT-4
llm_gpt4 = ChatOpenAI(
    model_name="gpt-4",
    openai_api_key=OPENAI_KEY,
    temperature=0,
    max_tokens=800
)

In [ ]:
# Optimized Retrieval Chain
qa = RetrievalQA.from_chain_type(
    llm=llm_gpt4,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.78
        }
    ),
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
    return_source_documents=True
)

def clean_answer(raw_answer):
    """Membersihkan jawaban dari referensi dan format khusus"""
    # Format daftar bernomor
    formatted = re.sub(r'(\d+\.)\s', r'\n\1 ', raw_answer)
    # Hapus karakter khusus
    cleaned = re.sub(r'[*_]{2}', '', formatted)
    return cleaned.strip()

def ask(query):
    try:
        # Proses query langsung tanpa validasi awal
        result = qa.invoke({"query": query})

        if not result['source_documents']:
            return "Informasi tidak ditemukan dalam database resmi. Silakan hubungi Kring Pajak 1500200"

        # Pembersihan jawaban akhir
        return clean_answer(result['result'])

    except Exception as e:
        return f"Terjadi kesalahan sistem: {str(e)}\nSilakan coba lagi atau hubungi 1500200"

faq_df = pd.read_csv('Dataset/faq_allobank_clean.csv')

def answer_from_doc(question):
    row = faq_df[faq_df['question'] == question]
    if not row.empty:
        return row.iloc[-1]['answer']
    else:
        return "Tidak ada jawaban."

# <b>Pertanyaan di dalam konteks Allo Bank</b>

In [9]:
# Test Cases 1
questions = [
    "Apa itu Allo Bank?",
    "Mengapa saya tidak bisa menggunakan Allo Pay saya?",
    "Mengapa saya tidak bisa log in di aplikasi Allo Bank?",
    "Apakah menyimpan dana di Allo Bank dipastikan aman?",
    "Mengapa aplikasi Allo Bank saya berjalan sangat lambat?",
    "Bagaimana jika saldo atau limit saya tidak cukup?",
    "Keuntungan apa yang saya dapatkan sebagai member Allo Bank?",
    "Mengapa saya tidak bisa mendaftarkan nomor ponsel/email di aplikasi Allo Bank?",
    "Biaya apa saja yang dikenakan ketika melakukan registrasi?",
    "Bagaimana cara membuat rekening Allo Bank?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Apa itu Allo Bank?
Jawaban:
Allo Bank adalah sebuah layanan perbankan berbasis digital yang dikembangkan oleh Allo Bank Indonesia. Layanan ini dirancang untuk memudahkan setiap nasabah dalam melakukan kegiatan finansial ataupun non finansial melalui satu aplikasi. Allo Bank, yang berarti "All in One", menawarkan konsep "one for all, and all for one", yang berarti semua untuk satu dan satu untuk semua. Allo Bank didirikan di Indonesia pada tahun 2021.

ACTUAL ANSWER: Sebuah layanan perbankan berbasis digital yang dapat memudahkan setiap nasabahnya dalam melakukan kegiatan finansial ataupun non finansial pada satu aplikasi. Bagi yang belum tahu, nama dari Allo Bank sebenarnya memiliki singkatan "All in One" dan mempunyai arti sebagai "one for all, and all for one" alias semua untuk satu dan satu untuk semua.

Pertanyaan: Mengapa saya tidak bisa menggunakan Allo Pay saya?
Jawaban:
Anda mungkin tidak dapat menggunakan Allo Pay Anda karena beberapa alasan berikut:
- Akun Anda b

# <b>Pertanyaan di luar Konteks Allo Bank</b>

In [10]:
# Test Cases 2
questions = [
    "Siapa presiden dan wakil presiden pertama Indonesia?",
    "Kapan Indonesia merdeka?",
    "Apakah alien itu benar-benar ada?",
    "Bagaimana cara belajar coding untuk pemula?",
    "Siapa penemu bom atom?",
    "Siapa itu Oppenheimer?",
    "Siapa itu Christoper Nolan?",
    "Apa itu teori relativitas?",
    "Apakah black hole itu nyata?",
    "Mengapa bumi itu bulat?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Siapa presiden dan wakil presiden pertama Indonesia?
Jawaban:
Maaf, sebagai asisten Allo Bank, saya hanya dapat memberikan informasi yang berkaitan dengan layanan dan produk Allo Bank. Untuk pertanyaan sejarah atau politik seperti ini, saya sarankan untuk mencarinya di sumber yang tepercaya. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Kapan Indonesia merdeka?
Jawaban:
Mohon maaf, pertanyaan Anda tidak relevan dengan layanan dan informasi yang disediakan oleh Allo Bank. Kami hanya dapat memberikan informasi terkait dengan layanan dan produk Allo Bank.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Apakah alien itu benar-benar ada?
Jawaban:
Maaf, pertanyaan Anda tidak relevan dengan layanan dan informasi yang disediakan oleh Allo Bank. Kami hanya dapat memberikan informasi terkait layanan dan produk Allo Bank.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Bagaimana cara belajar coding untuk pemula?
Jawaban:
Maaf, pertanyaan Anda tidak relevan dengan laya